In [1]:
# !mkdir ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# !kaggle datasets download -d manishkc06/startup-success-prediction
# !mkdir data
# !unzip startup-success-prediction.zip -d /content/data

# # !rm -r  ./data

In [2]:
import numpy as np
import pandas as pd 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('./data/startup data.csv')
# df = data.copy()

In [4]:
df.shape

(923, 49)

In [5]:
df.head()

,Unnamed: 0,state_code,latitude,longitude,zip_code,id,city,Unnamed: 6,name,labels,...,object_id,has_VC,has_angel,has_roundA,has_roundB,has_roundC,has_roundD,avg_participants,is_top500,status
0,1005,CA,42.358880,-71.056820,92101,c:6669,San Diego,NaN,Bandsintown,1,...,c:6669,0,1,0,0,0,0,1.0000,0,acquired
1,204,CA,37.238916,-121.973718,95032,c:16283,Los Gatos,NaN,TriCipher,1,...,c:16283,1,0,0,1,1,1,4.7500,1,acquired
2,1001,CA,32.901049,-117.192656,92121,c:65620,San Diego,San Diego CA 92121,Plixi,1,...,c:65620,0,0,1,0,0,0,4.0000,1,acquired
3,738,CA,37.320309,-122.050040,95014,c:42668,Cupertino,Cupertino CA 95014,Solidcore Systems,1,...,c:42668,0,0,0,1,1,1,3.3333,1,acquired
4,1002,CA,37.779281,-122.419236,94105,c:65806,San Francisco,San Francisco CA 94105,Inhale Digital,0,...,c:65806,1,1,0,0,0,0,1.0000,1,closed


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 923 entries, 0 to 922
Data columns (total 49 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                923 non-null    int64  
 1   state_code                923 non-null    object 
 2   latitude                  923 non-null    float64
 3   longitude                 923 non-null    float64
 4   zip_code                  923 non-null    object 
 5   id                        923 non-null    object 
 6   city                      923 non-null    object 
 7   Unnamed: 6                430 non-null    object 
 8   name                      923 non-null    object 
 9   labels                    923 non-null    int64  
 10  founded_at                923 non-null    object 
 11  closed_at                 335 non-null    object 
 12  first_funding_at          923 non-null    object 
 13  last_funding_at           923 non-null    object 
 14  age_first_

In [7]:
df.describe()

,Unnamed: 0,latitude,longitude,labels,age_first_funding_year,age_last_funding_year,age_first_milestone_year,age_last_milestone_year,relationships,funding_rounds,...,is_consulting,is_othercategory,has_VC,has_angel,has_roundA,has_roundB,has_roundC,has_roundD,avg_participants,is_top500
count,923.000000,923.000000,923.000000,923.000000,923.000000,923.000000,771.000000,771.000000,923.000000,923.000000,...,923.000000,923.000000,923.000000,923.000000,923.000000,923.000000,923.000000,923.000000,923.000000,923.000000
mean,572.297941,38.517442,-103.539212,0.646804,2.235630,3.931456,3.055353,4.754423,7.710726,2.310943,...,0.003250,0.322860,0.326111,0.254605,0.508126,0.392199,0.232936,0.099675,2.838586,0.809317
std,333.585431,3.741497,22.394167,0.478222,2.510449,2.967910,2.977057,3.212107,7.265776,1.390922,...,0.056949,0.467823,0.469042,0.435875,0.500205,0.488505,0.422931,0.299729,1.874601,0.393052
min,1.000000,25.752358,-122.756956,0.000000,-9.046600,-9.046600,-14.169900,-7.005500,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,283.500000,37.388869,-122.198732,0.000000,0.576700,1.669850,1.000000,2.411000,3.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.500000,1.000000
50%,577.000000,37.779281,-118.374037,1.000000,1.446600,3.528800,2.520500,4.476700,5.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,2.500000,1.000000
75%,866.500000,40.730646,-77.214731,1.000000,3.575350,5.560250,4.686300,6.753400,10.000000,3.000000,...,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,3.800000,1.000000
max,1153.000000,59.335232,18.057121,1.000000,21.895900,21.895900,24.684900,24.684900,63.000000,10.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,16.000000,1.000000


In [8]:
df.columns

Index(['Unnamed: 0', 'state_code', 'latitude', 'longitude', 'zip_code', 'id',
       'city', 'Unnamed: 6', 'name', 'labels', 'founded_at', 'closed_at',
       'first_funding_at', 'last_funding_at', 'age_first_funding_year',
       'age_last_funding_year', 'age_first_milestone_year',
       'age_last_milestone_year', 'relationships', 'funding_rounds',
       'funding_total_usd', 'milestones', 'state_code.1', 'is_CA', 'is_NY',
       'is_MA', 'is_TX', 'is_otherstate', 'category_code', 'is_software',
       'is_web', 'is_mobile', 'is_enterprise', 'is_advertising',
       'is_gamesvideo', 'is_ecommerce', 'is_biotech', 'is_consulting',
       'is_othercategory', 'object_id', 'has_VC', 'has_angel', 'has_roundA',
       'has_roundB', 'has_roundC', 'has_roundD', 'avg_participants',
       'is_top500', 'status'],
      dtype='object')

In [9]:
df['acquired'] = pd.get_dummies(df['status'])['acquired']

In [10]:
df = df.drop(['zip_code'],axis=1)
df = df.drop(['city'],axis=1)
df = df.drop(['id'],axis=1)
df = df.drop(['name'],axis=1)
df = df.drop(['Unnamed: 6'],axis=1)
df = df.drop(['category_code'],axis=1)
df = df.drop(['object_id'],axis=1)
df = df.drop(['founded_at'],axis=1)
df = df.drop(['closed_at'],axis=1)
df = df.drop(['first_funding_at'],axis=1)
df = df.drop(['last_funding_at'],axis=1)

df = df.drop(['Unnamed: 0'],axis=1)
df = df.drop(['latitude','longitude'],axis=1)

df = df.drop(['state_code'],axis=1)
df = df.drop(['relationships'],axis=1)
df = df.drop(['status'],axis=1)
df = df.drop(['state_code.1'],axis=1)
df = df.drop(['age_last_milestone_year'],axis=1)
df = df.drop(['age_first_milestone_year'],axis=1)

In [11]:
df.head()

,labels,age_first_funding_year,age_last_funding_year,funding_rounds,funding_total_usd,milestones,is_CA,is_NY,is_MA,is_TX,...,is_othercategory,has_VC,has_angel,has_roundA,has_roundB,has_roundC,has_roundD,avg_participants,is_top500,acquired
0,1,2.2493,3.0027,3,375000,3,1,0,0,0,...,1,0,1,0,0,0,0,1.0000,0,1
1,1,5.1260,9.9973,4,40100000,1,1,0,0,0,...,0,1,0,0,1,1,1,4.7500,1,1
2,1,1.0329,1.0329,1,2600000,2,1,0,0,0,...,0,0,0,1,0,0,0,4.0000,1,1
3,1,3.1315,5.3151,3,40000000,1,1,0,0,0,...,0,0,0,0,1,1,1,3.3333,1,1
4,0,0.0000,1.6685,2,1300000,1,1,0,0,0,...,0,1,1,0,0,0,0,1.0000,1,0


In [12]:
y = df['acquired']
X = df.drop(['acquired'],axis=1)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,y , random_state=24, shuffle=True, test_size=0.2)

In [14]:
print("Shape of the X Train :", X_train.shape)
print("Shape of the y Train :", y_train.shape)
print("Shape of the X test :", X_test.shape)
print("Shape of the y test :", y_test.shape)

Shape of the X Train : (738, 29)
Shape of the y Train : (738,)
Shape of the X test : (185, 29)
Shape of the y test : (185,)


In [15]:
adaBoostClassifier = AdaBoostClassifier()
adaBoostClassifier.fit(X_train, y_train)
adaBoostClassifier_pred =  adaBoostClassifier.predict(X_test)

In [16]:
print('\nconfusion matrix')
print(confusion_matrix(y_test, adaBoostClassifier_pred))
print('\naccuracy')
print(accuracy_score(y_test, adaBoostClassifier_pred))
print('\nclassification report')
print(classification_report(y_test, adaBoostClassifier_pred))
roc_auc_score(y_test, adaBoostClassifier_pred)



confusion matrix
[[ 69   0]
 [  0 116]]

accuracy
1.0

classification report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        69
           1       1.00      1.00      1.00       116

    accuracy                           1.00       185
   macro avg       1.00      1.00      1.00       185
weighted avg       1.00      1.00      1.00       185



1.0

In [17]:
randomForestClassifier = RandomForestClassifier()
randomForestClassifier.fit(X_train,y_train)
randomForestClassifier_pred = randomForestClassifier.predict(X_test)

In [18]:
print('\nconfusion matrix') 
print(confusion_matrix(y_test, randomForestClassifier_pred))

print('\naccuracy')
print(accuracy_score(y_test, randomForestClassifier_pred))

print('\nclassification report')
print(classification_report(y_test, randomForestClassifier_pred)) 
roc_auc_score(y_test, randomForestClassifier_pred)



confusion matrix
[[ 69   0]
 [  0 116]]

accuracy
1.0

classification report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        69
           1       1.00      1.00      1.00       116

    accuracy                           1.00       185
   macro avg       1.00      1.00      1.00       185
weighted avg       1.00      1.00      1.00       185



1.0